In [50]:
#Initialization for HateSpeechRecognition
import scipy as sp
import numpy as np
import pandas as pd
import sklearn
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /home/mufa/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/mufa/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/mufa/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [51]:

def clean_text(rgx_list, text):
    new_text = text
    for rgx_match in rgx_list:
        new_text = re.sub(rgx_match, '', new_text)
    return new_text

In [58]:

stopwords_list = stopwords.words('english')
#other_exclusions = ["#ff", "ff", "rt","!",":","...",".","-","&","?"]
#stopwords_list.extend(other_exclusions)

dataset = pd.read_csv("dataset/labeled_data.csv")

tweets = dataset.tweet

ps = nltk.stem.PorterStemmer()

#ps.stem(dataset)

#Text cleaning and tokenization, then stemming then POS tagging
filtered_tweets=[];
tweet_tags = [];
filtered_tweets_stemmed=[];

for tweet in tweets:
    tweet = clean_text(["http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+",".*@.*:","&#*\w*","@[\w\-]+","[^\w\s]"],tweet)
    tweet = tweet.lower()
    word_tokens = word_tokenize(tweet)
    filtered_tweets.append([word for word in word_tokens if not word in stopwords_list])
    filtered_tweets_stemmed.append([ps.stem(word) for word in word_tokens if not word in stopwords_list])
    
for tweet in filtered_tweets:
    tweet_tags.append(nltk.pos_tag(tweet))



[[('momma', 'NN'), ('said', 'VBD'), ('pussy', 'JJ'), ('cats', 'NNS'), ('inside', 'IN'), ('doghouse', 'NN')], [('hated', 'VBN'), ('hoes', 'NNS'), ('favorite', 'RB'), ('2mw', 'CD'), ('sevenone', 'NN')], [('nice', 'JJ'), ('girls', 'NNS'), ('bad', 'JJ'), ('make', 'VBP'), ('get', 'VB'), ('naughty', 'JJ'), ('bad', 'JJ'), ('yello', 'NN'), ('hoe', 'NN'), ('real', 'JJ'), ('nice', 'JJ'), ('body', 'NN'), ('south', 'JJ'), ('chick', 'NN'), ('like', 'IN'), ('em', 'JJ'), ('real', 'JJ'), ('thick', 'NN')], [('pimps', 'NNS'), ('pimps', 'NNS'), ('hoes', 'NNS'), ('future', 'JJ'), ('voice', 'NN')], [('post', 'NN'), ('picture', 'NN'), ('pussy', 'NN'), ('get', 'VBP'), ('200', 'CD'), ('likes', 'NNS')], [('pussy', 'JJ'), ('powerful', 'JJ'), ('drug', 'NN'), ('happyhumpday', 'NN')], [('quick', 'JJ'), ('piece', 'NN'), ('pussy', 'NN'), ('call', 'NN'), ('drive', 'NN')], [('running', 'VBG'), ('round', 'NN'), ('like', 'IN'), ('brand', 'NN'), ('new', 'JJ'), ('pussy', 'JJ'), ('thats', 'NNS'), ('bout', 'VBP'), ('get', '